In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import re
import time

In [17]:
API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
term = "(PBMC)+AND+(single-cell)"
url_search = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200"

In [19]:
url_search

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409&term=(PBMC)+AND+(single-cell)'

In [18]:
info = BeautifulSoup(requests.get(url_search).text, 'lxml')

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [19]:
info

<?xml version="1.0" encoding="UTF-8" ?><!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<html><body><esearchresult><count>7150</count><retmax>200</retmax><retstart>0</retstart><idlist>
<id>200246456</id>
<id>200233315</id>
<id>200229626</id>
<id>200233276</id>
<id>200233275</id>
<id>200232117</id>
<id>200237254</id>
<id>200237579</id>
<id>200237562</id>
<id>200189050</id>
<id>200185712</id>
<id>200239626</id>
<id>200215441</id>
<id>200229406</id>
<id>200241835</id>
<id>200241834</id>
<id>200243629</id>
<id>200239915</id>
<id>200239909</id>
<id>200239903</id>
<id>200218065</id>
<id>200232003</id>
<id>200232002</id>
<id>200212479</id>
<id>200242477</id>
<id>200228078</id>
<id>200242127</id>
<id>200235927</id>
<id>200235857</id>
<id>200227952</id>
<id>200214546</id>
<id>200231977</id>
<id>200224460</id>
<id>200224456</id>
<id>200175524</id>
<id>200175523</id>
<id>200175522</id>
<id>200227408</id>
<id>20023

In [20]:
len(info.find_all('id'))

200

In [19]:
# id info
id = '200189050'
id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id}&rettype=summary"
id_info = BeautifulSoup(requests.get(id_url).text, 'xml')

In [29]:
id_url

'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409&id=200215441'

In [20]:
id_info

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE eSummaryResult PUBLIC "-//NLM//DTD esummary v1 20041029//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20041029/esummary-v1.dtd">
<eSummaryResult>
<DocSum>
<Id>200189050</Id>
<Item Name="Accession" Type="String">GSE189050</Item>
<Item Name="GDS" Type="String"/>
<Item Name="title" Type="String">Molecular Dissection of Disease Heterogeneity in SLE</Item>
<Item Name="summary" Type="String">Systemic lupus erythematosus (SLE) affects 1 in 537 of African American (AA) women, which is &gt;2-fold more than European American (EA) women. AA patients also develop the disease at a younger age, have more severe symptoms, and a greater chance of early mortality. We used a multi-omics approach to uncover ancestry-specific immune alterations in SLE patients and healthy controls that may contribute to disease disparities. Cell composition, signaling, and epigenetics were evaluated by mass cytometry; droplet-based single cell transcriptomics and paired 

In [21]:
GED = 'GSE189050'
ged_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={GED}"
detailed_info = BeautifulSoup(requests.get(ged_url).text, 'html.parser')

In [22]:
detailed_info

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<style type="text/css">
      a { text-decoration: none; }
	.bordergc {background-color: #6699CC;}
	.bordergd {background-color: #B6C7E5;}
	.borderge {background-color: #EEF3FB;}
	.bordergf {background-color: #FFFFFF;}
	.bordergg {background-color: #CCCCCC;}
      .small8b { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#6633cc;
              }
      .small8db { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#4411aa;
              }

    </style>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="NCBI GEO Gene Expression Omnibus microarray oligonucleotide array SAGE" name="keywords"/>
<meta content="NCBI's Gene Expression Omnibus (GEO) is a public archive and resource for gene expression data." name="description"/>
<meta content="geo" name="ncbi_app"/>
<meta content="full" name="ncb

In [10]:
# define a getting web information function
def get_literature_id(term):
    API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
    # pdat means published date, 2020[pdat] means publised literatures from 2020/01/01 to 2020/12/31
    url_start = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&api_key={API_KEY}&term={term}"
    # f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&api_key={API_KEY}&term={term }+{year}[pdat]"
    time.sleep(0.5)
    info = BeautifulSoup(requests.get(url_start, timeout=(5, 5)).text, 'html.parser')
    time.sleep(0.5)
    # translate str to int
    year_published_count = int(info.find('count').text)
    
    id_list = [_.get_text() for _ in info.find_all('id')]

    for page in range(1, math.ceil(year_published_count/20)):
        url_page = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&api_key={API_KEY}&term={term}&retmax=20&retstart={page*20}'
        # 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&api_key={API_KEY}&term=pbmc+AND+single+cell+{year}[pdat]&retmax=20&retstart={page*20}'
        time.sleep(0.5)
        info_page = BeautifulSoup(requests.get(url_page, timeout=(5, 5)).text, 'html.parser')
        id_list += [_.get_text() for _ in info_page.find_all('id')]

    return id_list, year_published_count

def get_detailed_info(id_list):
    API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
    # PMID DOI PMCID Title Abstract Author_1st Affiliation_1st Journel Pulication_time
    extracted_info = []
    for batch in range(0, math.ceil(len(id_list)/20)):
        id_str = ",".join(id_list[batch*20: (batch+1)*20])
        detailed_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&api_key={API_KEY}&id={id_str}&rettype=medline&retmode=text'
        time.sleep(0.5)
        detailed_info = BeautifulSoup(requests.get(detailed_url, timeout=(5, 5)).text, 'html.parser')
        literature_as_line_list = detailed_info.text.split('\nPMID')[1:]

        for literature in literature_as_line_list:
            try:
                # PMID
                pmid = literature.split('- ')[1].split('\n')[0]
                # DOI
                if '[doi]' in literature:
                    doi = literature.split('[doi]')[0].split(' - ')[-1].strip()
                else:
                    doi = ""
                # PMCID
                if "PMC" in literature:
                    pmcid = literature.split('PMC -')[1].split('\n')[0].strip()
                else:
                    pmcid = ""
                # Title
                title = re.split(r'\n[A-Z]{2,3}\s', literature.split('TI  - ')[1])[0].replace("\n      ", "")
                if '\n' in title:
                    title = title.replace("\n      ", "")
                # Abstract
                abstract = literature.split('AB  - ')[1].split(' - ')[0].replace("\n      ", "").split('\n')[0]
                # Author_1st
                author = literature.split('FAU - ')[1].split('\n')[0]
                # Affiliation_1st
                tmp_affiliation = literature.split('FAU - ')[1]
                if "AD  - " in tmp_affiliation:
                    affiliation = tmp_affiliation.split('AD  - ')[1].replace("\n      ", "").strip('\n')
                else:
                    affiliation = ""

                # Journel
                journel = literature.split('TA  - ')[1].split('\n')[0]
                # Publication time
                publication_time = literature.split('SO  - ')[1].split(';')[0].split('. ')[1]
                if ':' in publication_time:
                    publication_time = publication_time.split(':')[0]
            except:
                print(id_str)

            extracted_info.append([pmid, doi, pmcid, title, abstract, author, affiliation, journel, publication_time])

    return extracted_info

In [12]:
extracted_info = []

term = "(%22peripheral+blood+mononuclear+cells%22)+AND+(single-cell)"

id_list, count = get_literature_id(term=term)
extracted_info += get_detailed_info(id_list)

print_info = pd.DataFrame(extracted_info, columns=['PMID', 'DOI', 'PMCID', 'Title', 'Abstract', 'Author_1st', 'Affiliations_1st', 'Journel', 'Publication_time'])


37425926,37414801,37393920,37380987,37377962,37373158,37372428,37363363,37361874,37359417,37353001,37349231,37340787,37323505,37289818,37283946,37275889,37275879,37264110,37262073
37258906,37257452,37256972,37251380,37235118,37228227,37223018,37214338,37201755,37192158,37191796,37185977,37173304,37163920,37163015,37160973,37155154,37146127,37144201,37140218


ConnectionError: HTTPSConnectionPool(host='eutils.ncbi.nlm.nih.gov', port=443): Max retries exceeded with url: /entrez/eutils/efetch.fcgi?db=pubmed&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409&id=37132178,37126452,37123249,37111549,37100864,37095455,37095364,37094424,37092851,37090715,37080999,37078407,37077913,37068896,37060557,37041166,37039643,37038777,37038287,37024794&rettype=medline&retmode=text (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f24c2f5b100>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [ ]:
API_KEY = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
id_str = '37393920,37380987,37377962,37373158,37372428,37363363,37361874,37359417,37353001,37349231,37340787,37323505,37289818,37283946,37275889,37275879,37264110,37262073,37258906,37257452'
detailed_url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&api_key={API_KEY}&id={id_str}&rettype=medline&retmode=text'
detailed_info = BeautifulSoup(requests.get(detailed_url, timeout=(5, 5)).text, 'html.parser')


In [11]:
count

854

In [12]:
print_info.shape

(851, 9)

In [13]:
print_info.to_csv('/mnt/c/Users/ouyangkang/Desktop/result_1030.csv', index=None)

In [4]:
data = pd.read_csv('/mnt/c/Users/ouyangkang/Desktop/result_1030.csv')
check = pd.read_csv('/mnt/c/Users/ouyangkang/Desktop/cxy-1031.csv')

In [6]:
check.head()

,DOI,Journal,Title,Date,Reported cells total,Organism,Tissue,Technique,Data location,Measurement,Disease,Number of reported cell types or clusters,H5AD location,Isolation,Number individuals,Cell source
0,10.1101/2020.01.10.902361,bioRxiv,Highly Multiplexed Single-Cell Full-Length cDN...,20200112,"3,000",Human,Blood,R2C2,PRJNA599962,RNA-seq,NaN,3.0,NaN,NaN,2.0,PBMCs
1,10.1038/s41467-019-14118-w,Nat Commun,Integrated single cell analysis of blood and c...,20200114,"65,326",Human,"Blood, Spine",Chromium,GSE138266,RNA-seq,Multiple sclerosis,17.0,NaN,Nanodroplet dilution,10.0,Cerebrospinal fluid
2,10.1101/2020.01.15.897066,bioRxiv,Sampling artifacts in single-cell genomics coh...,20200115,"66,136",Human,Blood,Chromium,NaN,RNA-seq,NaN,4.0,NaN,NaN,5.0,PBMCs
3,10.1172/jci.insight.134700,JCI Insight,Myocardial B cells are a subset of circulating...,20200116,NaN,Mouse,"Blood, Heart",Chromium,NaN,RNA-seq,NaN,NaN,NaN,NaN,NaN,NaN
4,10.1038/s41591-019-0733-7,Nat Med,Targeted therapy guided by single-cell transcr...,20200120,"54,476",Human,"Blood, Skin",Chromium,GSE132802,RNA-seq,DiH DRESS,NaN,NaN,NaN,6.0,"PBMCs, Skin biopsies"


In [7]:
data.head()

,PMID,DOI,PMCID,Title,Abstract,Author_1st,Affiliations_1st,Journel,Publication_time
0,37885831,10.1016/j.ibneur.2023.10.003,PMC10598524,Evaluation of DNA repair capacity in parents o...,BACKGROUND: Autism Spectrum Disorder (ASD) is ...,"Akouchekian, Mansoureh","Department of Medical Genetics, School of Medi...",IBRO Neurosci Rep,2023 Oct 19
1,37884975,10.1186/s12967-023-04497-6,PMC10605335,"HCG18, LEF1AS1 and lncCEACAM21 as biomarkers o...",BACKGROUND: Even after 3 years from SARS-CoV-2...,"Greco, Simona","Molecular Cardiology Laboratory, IRCCS Policli...",J Transl Med,2023 Oct 26
2,37881939,10.1161/ATVBAHA.123.318974,NaN,Single-Cell Transcriptomics Reveals Crucial Ce...,BACKGROUND: Carotid atherosclerosis is a chron...,"Tan, Jinyun","Department of Vascular Surgery, Huashan Hospit...",Arterioscler Thromb Vasc Biol,2023 Oct 26
3,37875396,10.1002/advs.202300123,NaN,Identification and Characterization of CD8(+) ...,Systemic Lupus Erythematosus (SLE) etiopathoge...,"Zhang, Lulu","Department of Rheumatology, Shanghai Jiao Tong...",Adv Sci (Weinh),2023 Oct 24
4,37873485,10.1101/2023.10.07.561349,PMC10592658,Mast-Cell Expressed Membrane Protein-1 (MCEMP1...,BACKGROUND: Mast-Cell Expressed Membrane Prote...,"Perrot, Carole Y",NaN,bioRxiv,2023 Oct 10


In [8]:
len(set(check['DOI'].tolist()) & set(data['DOI'].tolist()))

24

In [9]:
check.shape

(121, 16)

In [4]:
doi_data = data[~data['DOI'].isna()]

In [20]:
doi_list = doi_data["DOI"].tolist()
pmid_list = doi_data["PMID"].tolist()
for doi, pmid in zip(doi_list, pmid_list):
    download_url = f'https://sci.bban.top/pdf/{doi}.pdf?#view=FitH'
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"}
    literature = requests.get(download_url, headers=headers)
    if literature.status_code != 200:
        print(f"this paper may have not downloading permission, it's doi: {doi}")
    else:
        with open(f'/mnt/c/Users/ouyangkang/Desktop/scraper_literature/{pmid}.pdf', 'wb') as f:
            f.write(literature.content)


this paper may have not downloading permission, it's doi: 10.14218/JCTH.2017.00077
this paper may have not downloading permission, it's doi: 10.1371/journal.pone.0194593
this paper may have not downloading permission, it's doi: 10.1093/infdis/jiy086
this paper may have not downloading permission, it's doi: 10.1093/hmg/ddx413
this paper may have not downloading permission, it's doi: 10.1128/JVI.01308-19
this paper may have not downloading permission, it's doi: 10.1093/nar/gkz601
this paper may have not downloading permission, it's doi: 10.12688/f1000research.18533.1
this paper may have not downloading permission, it's doi: 10.1097/CM9.0000000000000306
this paper may have not downloading permission, it's doi: 10.7717/peerj.6757
this paper may have not downloading permission, it's doi: 10.1093/jac/dkz187
this paper may have not downloading permission, it's doi: 10.1039/c8sc04407d
this paper may have not downloading permission, it's doi: 10.1093/ibd/izz039
this paper may have not downloadi